In [ ]:
import sys
import os
import random
import glob
import ngsci
import h5py
import cv2
import yaml
import matplotlib
import torch
#import hydra
import numpy as np
import pandas as pd
from tqdm import tqdm
from openslide import OpenSlide
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from omegaconf import DictConfig
import h5py

#### function to read individual files each with one sample's data

In [ ]:
def load_h5_file(filename):
    with h5py.File(filename, "r") as f:
        coords = f['coords'][()]
        features = f['features'][()] #[:,0]#[()]
        tb_positive = f['tb_positive'][()]
        
        return coords, features, tb_positive

#### here locate correct config

In [ ]:
conf_preproc = OmegaConf.load("/home/ngsci/project/tuberculosis_detection/conf/preproc.yaml")
#conf_preproc = conf_preproc["transformer_on_embeddings_bag"]["dinov2-vit-small_224_224_patches"]
conf_preproc = conf_preproc["transformer_on_embeddings_bag"]["uni_224_224_patches"]

#### try out what comes from config

In [ ]:
tb_labels = pd.read_csv(conf_preproc["tb_labels_csv"])

In [ ]:
conf_preproc["emb_dir"]

In [ ]:
file_names_all_h5 = np.array( np.array( sorted(  glob.glob( os.path.join(conf_preproc["emb_dir"], "*.h5"))   )) )

In [ ]:
file_names_all_h5[:3]

In [ ]:
file_names_all_h5[0], file_names_all_h5.shape

In [ ]:
coordst, featurest, tb_positivet = load_h5_file(file_names_all_h5[0])
coordst.shape, featurest.shape, tb_positivet

In [ ]:
BAG_DIM = featurest.shape[0]
EMB_DIM = featurest.shape[1]
#((110, 2), (110, 384), 0) or other possible dims for embeddings

features_all = np.zeros((file_names_all_h5.shape[0], BAG_DIM, EMB_DIM), dtype=np.float16)
tb_positive_all = np.zeros(file_names_all_h5.shape[0])
coords_all = np.zeros((file_names_all_h5.shape[0], BAG_DIM, 2), dtype=np.uint16)

for f in tqdm(range(file_names_all_h5.shape[0])):
    coords, features, tb_positive = load_h5_file(file_names_all_h5[f])
    features_all[f] = features
    tb_positive_all[f] = tb_positive
    coords_all[f] = coords
    

#### check if save path is CORRECT !!

In [ ]:
conf_preproc["emb_h5"]

#### saving

In [ ]:
%%time
with h5py.File(conf_preproc["emb_h5"], 'w') as h5f:
    h5f.create_dataset("features", data=features_all)
    h5f.create_dataset("coords", data=coords_all)
    h5f.create_dataset("tb_positive", data=tb_positive_all)

### TESTING

In [ ]:
def load_h5_file_test(filename):
    with h5py.File(filename, "r") as f:
        coords = f['coords'][()]
        features = f['features'][()]
        tb_positive = f['tb_positive'][()]
        
        return coords, features, tb_positive

In [ ]:
coordst, featurest, tb_positivet = load_h5_file_test(conf_preproc["emb_h5"])

In [ ]:
coordst.shape, featurest.shape, tb_positivet.shape

In [ ]:
featurest.min(), featurest.max(), featurest.mean()